In [23]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 5.5 MB/s eta 0:00:00


In [1]:
# Import libraries
from transformers import pipeline # Load pre-trained NLP models easily
import pandas as pd # Handle tabular data (DataFrames)
import re  # Text cleaning and pattern matching (Regex- Regular Expressions)

In [2]:
# Load dataset
df = pd.read_csv("121conversations.csv")
df.head(2)

,callid,channel,transcript
0,12fbf009-dbd6-46de-bac4-f35bbc043039,agent,Hello! Thank you for reaching out to Digital W...
1,12fbf009-dbd6-46de-bac4-f35bbc043039,customer,Hi Kevin! I'm experiencing some trouble with m...


In [3]:
def clean_text(text):
    text = re.sub(r'[,"“”‘’\'\!]', '', text) # Remove punctuation marks
    text = text.strip().lower() # Trim spaces & convert to lowercase
    return text

df['transcript'] = df['transcript'].astype(str).apply(clean_text)

In [4]:
df.head(2)

,callid,channel,transcript
0,12fbf009-dbd6-46de-bac4-f35bbc043039,agent,hello thank you for reaching out to digital wo...
1,12fbf009-dbd6-46de-bac4-f35bbc043039,customer,hi kevin im experiencing some trouble with my ...


In [5]:
# Aggregate Agent Utterances per Call ID for agent name extraction
Agent_utterances_df = df[df['channel'] == 'agent'].groupby('callid')['transcript'].apply(lambda x: ' '.join(x)).reset_index()
Agent_utterances_df.rename(columns={'transcript': 'all_agent_utterances'}, inplace=True)

In [6]:
# Agent_utterances_df.to_csv('Agent_utterances_df.csv', index=False) # Checked data correct or not

In [7]:
def extract_agent_name(text):
    patterns = [
        r"my name is ([a-z]+)",
        r"this is ([a-z]+)",
        r"you're speaking with ([a-z]+)"
    ]
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            return match.group(1).capitalize()
    return None


# Extract agent names from aggregated utterances
Agent_utterances_df['agent_name'] = Agent_utterances_df['all_agent_utterances'].apply(extract_agent_name)

# # Now map back to original df based on 'callid'
# df = df.merge(Agent_utterances_df[['callid', 'agent_name']], on='callid', how='left')

In [8]:
Agent_utterances_df.head(3)

,callid,all_agent_utterances,agent_name
0,07669c87-d6a6-47ff-aaab-b697b854f045,hello thank you for calling digital workspace ...,Lisa
1,07fbf9cf-b317-4753-ae9b-5eb6c5eb0756,good day thank you for contacting digital work...,Michael
2,091a69a0-cd55-4b33-b618-74b56f007af9,im sorry to hear that. ill be glad to help you...,None


In [9]:
Agent_utterances_df.value_counts('agent_name')

,count
agent_name,
Emily,36
Sarah,8
Michael,7
Alex,5
Jessica,4
Lisa,4
James,4
Chris,2
Jennifer,1


In [10]:
# Aggregate Customer Utterances per Call ID for call-level sentiment/summary
# customer_utterances_df = df[df['channel'] == 'customer'].groupby('callid')['transcript'].apply(lambda x: ' '.join(x)).reset_index()
# customer_utterances_df.rename(columns={'transcript': 'all_customer_utterances'}, inplace=True)
import re

# Get first 2 sentences split by ., ?, or /
def get_first_two(text):
    parts = re.split(r'[.?/]+', text)
    parts = [p.strip() for p in parts if p.strip()]
    return '. '.join(parts[:2]) + '.' if parts else ''

# Aggregate and keep only first 2 sentences
customer_utterances_df = (
    df[df['channel'] == 'customer']
    .groupby('callid')['transcript']
    .apply(lambda x: get_first_two(' '.join(x)))
    .reset_index(name='all_customer_utterances')
)


In [11]:
# Regex Dissatisfaction Detection
dissatisfaction_keywords = [
    r"not happy", r"unhappy", r"angry", r"frustrated", r"disappointed", "sad", "bad",
    r"bad experience", r"terrible", r"upset", r"annoyed", r"unacceptable", "damage", "damaged"
]

def detect_dissatisfaction_regex_keywords(text):
    found_keywords = []
    if not isinstance(text, str):
        return ''
    for word in dissatisfaction_keywords:
        if re.search(word, text, re.IGNORECASE):
            found_keywords.append(word)
    return ", ".join(found_keywords) if found_keywords else ''

# Extract agent names from aggregated utterances
customer_utterances_df['EOD'] = customer_utterances_df['all_customer_utterances'].apply(detect_dissatisfaction_regex_keywords)

In [12]:
customer_utterances_df.head(20)

,callid,all_customer_utterances,EOD
0,07669c87-d6a6-47ff-aaab-b697b854f045,hi lisa i want to create a new account for our...,
1,07fbf9cf-b317-4753-ae9b-5eb6c5eb0756,hi michael i recently joined the company and i...,
2,091a69a0-cd55-4b33-b618-74b56f007af9,hi im trying to find my invoice but i cant see...,
3,09fbdf5c-f563-46cd-87fd-449fa2b58563,hi emily weve been using your digital workspac...,
4,0cefb51a-9dac-48d3-ae2f-aa90cb2773b2,good afternoon i need to get a copy of my invo...,
5,11777617-c2a7-412c-914c-44ffc9746d2b,hi id like to place an order for a new smartph...,
6,11a65e3e-7654-4d35-9ddd-45fb20a1108b,hi emily im having trouble resetting my passwo...,
7,11a9a4d7-3250-44cb-83eb-9f12d3ef487f,hi emily i recently purchased a product from y...,
8,12fbf009-dbd6-46de-bac4-f35bbc043039,hi kevin im experiencing some trouble with my ...,
9,14b8d826-842a-4465-847e-d3b5b61fa953,hi emily i ordered a product a week ago and th...,


In [13]:
# Regex Complaint Detection
complaint_keywords = [r"complaint", r"complain"]

def detect_complaint(text):
    for word in complaint_keywords:
        if re.search(word, text):
            return True
    return False
# detect if the call is a complaint call
customer_utterances_df['is_complaint_call'] = customer_utterances_df['all_customer_utterances'].apply(detect_complaint)

In [14]:
customer_utterances_df.head(2)

,callid,all_customer_utterances,EOD,is_complaint_call
0,07669c87-d6a6-47ff-aaab-b697b854f045,hi lisa i want to create a new account for our...,,False
1,07fbf9cf-b317-4753-ae9b-5eb6c5eb0756,hi michael i recently joined the company and i...,,False


In [15]:
# Regex Regulatory (Regulatory call is defined as a call related to customer's monetary issues) Detection
money_keywords = [r"money", r"refund", r"payment", r"charged", r"billing",
                  r"invoice", r"credit card", r"debit", r"balance", r"account statement"]

def detect_money_issue(text):
    for word in money_keywords:
        if re.search(word, text):
            return True
    return False

# Detect if money-related terms are mentioned
customer_utterances_df['mentions_money'] = customer_utterances_df['all_customer_utterances'].apply(detect_money_issue)

# A regulatory call is one that is both a complaint and mentions money
customer_utterances_df['is_regulatory'] = customer_utterances_df.apply(
    lambda row: row['is_complaint_call'] and row['mentions_money'],
    axis=1
)

In [16]:
## Note: I worked on Regulatory calls real time and therefore tried to showcase a simple way to do this task. In this dataset I saw only one such regulatory call.


In [17]:
# Text Summarization (on combined customer utterances)
SUMMARIZATION_MODEL_NAME = "google/flan-t5-small" # Used "google/flan-t5-large"- (too large to run on local), "sshleifer/distilbart-cnn-12-6"- poorly performed
summarizer = pipeline("summarization", model=SUMMARIZATION_MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


In [18]:
# customer_utterances_df.to_csv('customer_utterances_df.csv', index=False) # Checked data correct or not

In [19]:
%%time
def get_customer_focused_summary(text, max_len=10, min_len=5):
    if not text or not isinstance(text, str):
        return None
    try:
        prompt = prompt = f"""Task: Extract only the customer's core intent or issue the text.

Instructions:
- Extract only the main problem, request, or complaint the customer expressed.
- Ignore personal details.
- Focus only on the actual issue raised.

Text: {text}"""


        summary = summarizer(prompt, max_length=max_len, min_length=min_len, do_sample=False)
        return summary[0]['summary_text'].strip() # .strip() removes leading/trailing whitespace

    except Exception as e:
        print(f"Error during summarization: {e} for text: {text[:100]}...")
        return "Summarization failed for this entry."

print("Summarization")
# Apply the new summary function to the combined customer utterances
customer_utterances_df['customer_issue_summary'] = customer_utterances_df['all_customer_utterances'].apply(get_customer_focused_summary)

Summarization


Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both

CPU times: user 45.4 s, sys: 514 ms, total: 45.9 s
Wall time: 46.2 s


In [20]:
customer_utterances_df.head(10)

,callid,all_customer_utterances,EOD,is_complaint_call,mentions_money,is_regulatory,customer_issue_summary
0,07669c87-d6a6-47ff-aaab-b697b854f045,hi lisa i want to create a new account for our...,,False,False,False,I want to create a new account for our company.
1,07fbf9cf-b317-4753-ae9b-5eb6c5eb0756,hi michael i recently joined the company and i...,,False,False,False,Can you help me recover my password?
2,091a69a0-cd55-4b33-b618-74b56f007af9,hi im trying to find my invoice but i cant see...,,False,True,False,email address is customer@example.
3,09fbdf5c-f563-46cd-87fd-449fa2b58563,hi emily weve been using your digital workspac...,,False,False,False,hi emily weve been using your digital workspac...
4,0cefb51a-9dac-48d3-ae2f-aa90cb2773b2,good afternoon i need to get a copy of my invo...,,False,True,False,I need to get a copy of my invoice for my rece...
5,11777617-c2a7-412c-914c-44ffc9746d2b,hi id like to place an order for a new smartph...,,False,False,False,i want to order the latest abc model.
6,11a65e3e-7654-4d35-9ddd-45fb20a1108b,hi emily im having trouble resetting my passwo...,,False,False,False,i haven't received any email with the link
7,11a9a4d7-3250-44cb-83eb-9f12d3ef487f,hi emily i recently purchased a product from y...,,False,True,False,I recently purchased a product from your websi...
8,12fbf009-dbd6-46de-bac4-f35bbc043039,hi kevin im experiencing some trouble with my ...,,False,False,False,I'm experiencing some trouble with my computer.
9,14b8d826-842a-4465-847e-d3b5b61fa953,hi emily i ordered a product a week ago and th...,,False,False,False,no sign of the package yet


In [21]:
# For categorizing the issues of each call
from transformers import pipeline
# Zero-shot classification pipeline
# This model is designed for zero-shot classification based on NLI
classifier = pipeline("zero-shot-classification",
                      model="typeform/distilbert-base-uncased-mnli",
                      device="cuda")

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/258 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Device set to use cuda


In [22]:
%%time

candidate_labels = ["login issue", "password recovery", "billing inquiry",
                    "technical support", "account update", "general query",
                    "internet connectivity", "computer freezing issue",
                    "subscription change", "order status", "refund request",
                    "service cancellation", "VPN connectivity issue", "delivery enquiry",
                    "shipping address enquiry", "Email trouble"]

def get_issue_category_zero_shot(text, labels=candidate_labels):
    if not text or not isinstance(text, str):
        return "No text provided."
    try:
        result = classifier(text, labels, multi_label=False)
        return result['labels'][0]
    except Exception as e:
        # Catching potential tokenization errors for very short/bad input
        print(f"Error during zero-shot classification: {e} for text: {text[:100]}...")
        return "Classification failed for this entry."

customer_utterances_df['issue_category_zeroshot'] = customer_utterances_df['all_customer_utterances'].apply(get_issue_category_zero_shot)

customer_utterances_df.head()

CPU times: user 9.91 s, sys: 47.3 ms, total: 9.96 s
Wall time: 10 s


,callid,all_customer_utterances,EOD,is_complaint_call,mentions_money,is_regulatory,customer_issue_summary,issue_category_zeroshot
0,07669c87-d6a6-47ff-aaab-b697b854f045,hi lisa i want to create a new account for our...,,False,False,False,I want to create a new account for our company.,account update
1,07fbf9cf-b317-4753-ae9b-5eb6c5eb0756,hi michael i recently joined the company and i...,,False,False,False,Can you help me recover my password?,login issue
2,091a69a0-cd55-4b33-b618-74b56f007af9,hi im trying to find my invoice but i cant see...,,False,True,False,email address is customer@example.,Email trouble
3,09fbdf5c-f563-46cd-87fd-449fa2b58563,hi emily weve been using your digital workspac...,,False,False,False,hi emily weve been using your digital workspac...,general query
4,0cefb51a-9dac-48d3-ae2f-aa90cb2773b2,good afternoon i need to get a copy of my invo...,,False,True,False,I need to get a copy of my invoice for my rece...,delivery enquiry


In [25]:
# save output files and input file
file_name = 'Smart_Agent_Assist_final_output1.0.xlsx'

with pd.ExcelWriter(file_name, engine='xlsxwriter') as writer:
    # Save input df to 'Sheet1'
    df.to_excel(writer, sheet_name='Input_File', index=False)

    # Save Agent_utterances_df (per-call analysis) to 'Sheet2'
    Agent_utterances_df.to_excel(writer, sheet_name='Agent_name_Entity', index=False)

    # Save customer_utterances_df (per-call analysis) to 'Sheet3'
    customer_utterances_df.to_excel(writer, sheet_name='cust_Ent_summ_categ', index=False)

print(f"saved {file_name}")

saved Smart_Agent_Assist_final_output1.0.xlsx
